## Classy classification

Classy classification - библиотека, в которой содержатся модели для классификации текстов. 

### Установка

In [3]:
!pip install classy-classification
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=843e208bb4e49040f5a87f28bb1a8da55fc855ace403e51dc20917a7c07a4397
  Stored in directory: /root/.cache/pip/wh

### Импорты

In [42]:
from random import randint
from pprint import pprint

import matplotlib.pyplot as plt
import classy_classification
import seaborn as sns
import pandas as pd
import numpy as np
import spacy 
import pickle

In [5]:
with open('/content/contract_enforcement.pkl', 'rb') as fp:
  contract_enforcement = pickle.load(fp)
contract_enforcement = pd.Series(contract_enforcement)

contract_enforcement = contract_enforcement[~contract_enforcement.isna()]
contract_enforcement = contract_enforcement.values

In [6]:
with open('/content/garantee_enforcement.pkl', 'rb') as fp:
  garantee_enforcement = pickle.load(fp)
garantee_enforcement = pd.Series(garantee_enforcement)

garantee_enforcement = garantee_enforcement[~garantee_enforcement.isna()]
garantee_enforcement = garantee_enforcement.values

In [7]:
df = pd.read_json('/content/train_with_splitted_texts.json', encoding='utf-8')

In [8]:
df.head()

,id,text,label,target
0,809436509,[Извещение о проведении открытого конкурса в э...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...
1,854885310,[ЗАЯВКИ участника запроса котировок в электрон...,обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...
2,4382157,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%
3,184555082,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%
4,211645258,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%


### построение модели

попробуем построить модель для "обеспечение исполнения контракта"

In [9]:
data={
    'обеспечение исполнения контракта': list(set(contract_enforcement)),
    '1': ['ghbdtn', 'd']}
nlp = spacy.blank('ru')
nlp.add_pipe(
    'text_categorizer',
    config={
        'data': data,
        'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
        'device': 'gpu'
    }
)

In [11]:
query = 'Рандомный текст для теста.'
print(nlp(query)._.cats)


{'обеспечение исполнения контракта': 0.4785072867996663, '1': 0.5214927132003337}


In [12]:
query = 'Размер обеспечения исполнения Контракта составляет 5 (пять) процентов цены контракта'
print(nlp(query)._.cats)


{'обеспечение исполнения контракта': 6.858603499849512e-06, '1': 0.9999931413965002}


In [59]:
def choose_minimum_rate_sentence(result:dict):
  min_val = 1e4
  for key, value in result.items():
    if value < min_val:
      min_val = value
      sentence = key
  return sentence, min_val

In [75]:
result = {}
string_num = randint(1, 300)
for sentence in df.loc[string_num, 'text']:
  result[sentence] = nlp(sentence)._.cats['обеспечение исполнения контракта']

pred = choose_minimum_rate_sentence(result)
print(pred)
print(df.loc[string_num, 'target'])
print(pred[0] == df.loc[string_num, 'target'])

('Обеспечение исполнения настоящего контракта установлено в размере 417,88, эквивалентном 0,50 % от начальной (максимальной) цены контракта [footnoteRef:16]  [16:', 4.459174911524309e-06)
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
False


Видим, что чем меньше полученное значение, тем ближе наш текст по смыслу

Получили отличные результаты, осталось разделить на трейн/тест и посчитать значение accuracy